# Fast Multipole Methods

Fast Multipole Methods allow the $\mathcal{O}(N)$ computation of Green's function interactions. While the algorithmic description is very different to $\mathcal{H}$-matrix methods the underlying idea of evaluating near-field interactions directly and far-field interactions approximately through low-rank representations, is similar.

There are a multitude of different types of Fast Multipole Methods. In this section we give a very high-level description that avoids technical details as far as possible which distinguish different typs of FMM.

We assume that we have $N$ particles $x_i$, $i=1, \dots, N$ in $[0, 1]^d$ for $d=2,3$. Let $G$ be the matrix of Green's function interactions defined by $G[i, j] = g(x_i, x_j)$. Hence, we assume that we have the same set of source and target particles. This is no restriction.

For a given vector $w$ we want to evaluate the product $Gw$.

As a first step we setup an octree/quadtree over $[0, 1]^3$. As opposed to the earlier description, where we used an adaptive tree for simplicity we will setup a regular tree of depth $L$. The root is level zero.

For a quadtree this means that we $4^L$ leaf nodes at the finest level $L$.

## From particles to multipole expansions

Let $\tau$ be a leaf node. Denote by $x_{i_{\ell}}$, $\ell=1,\dots, n_{\tau}$ the $n_{\tau}$ particles associated with $\tau$, and by $B_{\tau}$ the octree/quadtree box associated with $\tau$.

Each particle generates a field $u_{\ell}^{\tau}(x) := w_{i_{\ell}}g(x, x_{i_{\ell}})$ that is well defined and smooth in $\mathbb{R}^3\backslash\overline{B_{\tau}}$. The factor $w_{i_{\ell}}$ is the corresponding element of our vector $w$. These factors are typically called particle weights.

The total field contribution of the box $B_{\tau}$ is $u_{\tau}(x) := \sum_{i_{\ell}}w_{i_{\ell}}g(x, x_{i_{\ell}})$.

The first operation in the FMM is the $P2M$ operation (Particle-to-Multipole), which approximates $u_{\tau}$ in the exterior of $B_{\tau}$ by a simpler representation $\tilde{u}_{\tau} := (P2L)u_{\tau}$. We do not specify here the algorithmic details but will discuss different types of $P2M$ operators later. The name Multipole goes back to the first FMM implementations and refers to an approximation of $u_{\tau}$ that is valid in the exterior of $B_{\tau}$.

## Multipole-To-Multipole Operation

We now traverse up the octree/quadtree. Let $\tau_{j}$ be a node on level $1 < j < L$. with box $B_{\tau_j}$. The field $u_{\tau_j}$, valid in the exterior of $B_{\tau_j}$ is defined by

$$
u_{\tau_j}(x) := \sum_{\tau'\in\mathcal{C}(\tau_j)} (M2M)u_{\tau'}(x).
$$

Here, $M2M$ is an operator that maps fields valid in the exterior of a box to a field valid in the exterior of the parent box. The implementation of the $M2M$ operator depends on the realisation of the FMM.

## The Multipole-To-Local and Local-To_Local map

At this stage we have assigned exterior fields to all boxes in the tree from level to onwards to the leafs. These are the fields generated by the source particles. Now the second phase of the FMM starts in which we evaluate the fields generated by the sources at the targets. Here, we assumed that sources and targets are identical. But this needs not be the case.

To understand the $M2L$ (Multipole-To-Local) map we first need to introduce the notion of interaction list.

For each node $\tau$ of the tree the interaction list consists of the children of the neighbours of the parent of $\tau$ that are not themselves direct neighbours of $\tau$. We denote the interaction list of $\tau$ by $\mathcal{I}(\tau)$.

In addition, we denote by $\mathcal{P}(\tau)$ the parent of $\tau$.

Let $B_{\tau}$ the box associated with $\tau$, 

We define the local field $u_{\tau}^{local}$ by

$$
u_{\tau}^{local} := (L2L)u_{\mathcal{P}(\tau)}\sum_{\tau' \in \mathcal{I}(\tau)}(M2L)u_{\tau}.
$$

The operator $(L2L)$ is a local-to-local map, meaning it maps a local field valid in the parent box to a local field valid in a child box. The L2L takes into account field contributions that have been computed at a lower level of the tree. This is added up to the contributions of all the fields generated by boxes in the interaction list.

This map creates local field from the second level up down to the leaf level. On level 1 there is no interaction list yet and no contribution from the root.


## Local-To-Particle Map

This is the final step in the FMM. We have computed from the top of the tree down to the leafs all local fields generated from the interaction list and parent contributions.

Finally, at the leaf-level we compute for each box $\tau$ the field values at the leaf particles $x_{i_\ell}\in B_{\tau}$. We have two contributions, namely from the local field $u_{\tau}^{local}$ and from the fields at the leafs located by the particles in the neighbouring boxes. We denote the neighbours of $B_{\tau}$ by $\mathcal{N}(\tau)$. Hence, the total field value is given as

$$
u_{\tau}^{local}(x_{i_\ell}) = \left[(L2P)u_{\tau}^{local}\right](x_{i_\ell}) + \sum_{\tau'\in \mathcal{N}(\tau)}\sum_{i_\tau'}g(x_{i_{\ell}}, x_{i_\tau'}).
$$

The $L2P$ operator is usually a simple direct evaluation of fields from the representation given in a local basis.

## Summarizing the FMM flow

Let us summarize the FMM Steps.

- Phase 1 (Upward traversal of tree)
    - Compute exterior field approximations in each leaf box (P2M)
    - From the leaf up to level 2 combine the exterior fields of the children boxes
      into an exterior field approximation of the parent box (M2M)
- Phase 2 (Downward traversal of tree)
    - From level 2 onwards down to the leaf level compute local fields
      by summing up the contributions from the interaction list and from the local
      field of the parents (M2L + L2L)
    - On the leaf level evaluate the local fields at the target particle positions and
      add the contributions from the direct interactions with particles in neighbouring boxes (L2P + P2P)

## Complexity of the FMM

We note that the total number of boxes in an octree/quadtree is $\mathcal{O}(N)$.

In phase 1 in each leaf box we have to convert the fields generated by the source particles into exterior field expansions for each box. The cost of $\mathcal{O}(N)$ since we have $N$ particles.

We then use $M2M$ operators to pass the fields upwards. For each box in level $2$ and above this is one translation operation, hence a cost of $\mathcal{O}(N)$.

In phase 2 we traverse the tree downwards from level 2 to the leafs. The number of boxes in the interaction list is constant, independent of $N$. Hence, again we have a cost of $\mathcal{O}(N)$.

Finally, at the leafs we compute the evaluation of the local fields at the particles and add in the direct neighbour contributions. Again, this is a cost of $\mathcal{O}(N)$.

Hence, the full cost of the FMM is $\mathcal{O}(N)$. The above estimate however only works if the cost of the translation operators is independent of $N$. Here, the low-rank expansions come into play.

We recall that separable expansions of a given degree are the continuous equivalent of low-rank expansions. Analytic FMM methods use separable expansions into suitable basis functions of fixed degree $k$ to represent the translation operators. This degree $k$ is chosen independently of $N$. Algebraic FMM methods often use mixtures of interpolation/approximation and matrix low-rank techniques to compute translation operators with a fixed rank $k'$ that is also independent of $N$.

## Types of FMM implementations

We have so far described the general flow of every FMM realisation. The difference between implementations is in how the various translation operators are implemented.

Frequently used implementations include the following.

- The classical Greengard/Rohklin FMM based on spherical harmonics expansions. This is the classical FMM as originally described by Greengard and Rohklin. It is based on approximating fields by spherical harmonic expansions and the translation of these expansions between boxes.
Over the years a number of improvements have been made to these methods. They are memory efficient and fast.

- Kernel-Independent FMM. The KIFMM approximates local and exterior fields by discrete single-layer potential representations over the boundary of a box. Translation operators are implemented by a mixture of approximation operators, low-rank approximations, and Fast Fourier Transforms, depending on the implementational details. These methods are considered very efficient for high accuracy requirements.

- Black-Box FMM. The Black-Box FMM uses Chebychev interpolation to represent kernels. Translation operators are implemented by a mixture of low-rank approximation techniques and direct evaluation of Chebychev interpolation polynomials.